# 순환 신경망으로 IMDB 리뷰 분류하기

* 필요한 라이브러리 임포트

In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras


## IMDB 리뷰 데이터셋

In [ ]:
# 훈련세트, 테스트 세트 크기 확인


In [ ]:
# 첫 번째 리뷰의 길이 출력


In [ ]:
# 두 번째 리뷰의 길이 출력


In [ ]:
# 첫 번째 리뷰에 담긴 내용과 레이블 확인


In [ ]:
# 타겟 데이터 확인 긍정:1  부정: 0


In [ ]:
# 훈련 세트와 검증 세트 분리


In [ ]:
# 훈련 세트의 각 리뷰별 단어 길이 확인 및 평균 , 중간값 확인


In [ ]:
# 리뷰 길이 데이터의 분포 확인


### 리뷰의 길이를 일정하게 조정 - pad_sequence()

# 순환 신경망 만들기

## Model1 : One-Hot Encoding을 사용하여 RNN 입력 시퀀스 구성한 경우

In [ ]:
# One-Hot Encoding


In [ ]:
#원핫 인코딩이 잘 된 상태


## Model2. 단어 임베딩을 사용하여 RNN 입력 시퀀스 구성한 경우
- 각각의 단어(토큰)를 고정된 크기의 실수 벡터로 변환
- keras.layers.Embedding 클래스 활용

# LSTM RNN으로 IMDB  텍스트 분류

## RNN 순환층에 DropOut 적용하기

## 2개의 RNN층이 연결된 모델 구현하기

# GRU 모델 구현하기